## Сервис чтения книг

### Чтение данных

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [13]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
con=engine.connect()

### Исследование таблиц

In [14]:
# Функция вывода результатов SQL запроса
def q(query):
    display(pd.io.sql.read_sql(sql=text(query), con = con))

# Вывод данных таблицы books
query=('''
SELECT *
FROM books
LIMIT 5 
''')

q(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [15]:
# Вывод данных таблицы authors

query=('''
SELECT *
FROM authors
LIMIT 5 
''')

q(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [16]:
# Вывод данных таблицы publishers

query=('''
SELECT *
FROM publishers
LIMIT 5 
''')

q(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [17]:
# Вывод данных таблицы ratings

query=('''
SELECT *
FROM ratings
LIMIT 5 
''')

q(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [18]:
# Вывод данных таблицы reviews

query=('''
SELECT *
FROM reviews
LIMIT 5 
''')

q(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Вывод:**  
Прочитанные данные соответствуют заявленной в задании структуре

### Задания

**1. Посчитайте, сколько книг вышло после 1 января 2000 года**

In [19]:
query=('''
SELECT count(*) 
FROM books 
WHERE cast(publication_date AS date)>'2000-01-01' 
''')

q(query)

,count
0,819


`После 1 января 2000г вышло 819 книг.`

**2. Для каждой книги посчитайте количество обзоров и среднюю оценку**

In [20]:
query = '''
SELECT b.title, b.book_id,
       ROUND(AVG(ra.rating), 2) AS avg_rating,
       COUNT(DISTINCT re.review_id) AS total_reviews
FROM books b
JOIN ratings ra ON b.book_id = ra.book_id
LEFT JOIN reviews re ON b.book_id = re.book_id
GROUP BY b.title, b.book_id
ORDER BY total_reviews DESC
'''
q(query)

,title,book_id,avg_rating,total_reviews
0,Twilight (Twilight #1),948,3.66,7
1,Water for Elephants,963,3.98,6
2,The Glass Castle,734,4.21,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.41,6
4,The Curious Incident of the Dog in the Night-Time,695,4.08,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.67,0
996,The Natural Way to Draw,808,3.00,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.00,0
998,Essential Tales and Poems,221,4.00,0


`"Twilight" - книга, которая имеет наибольшее количество обзоров (7 обзоров)`

**3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры**

In [21]:
query = '''
SELECT p.publisher,
       COUNT(b.book_id) AS books
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY books DESC
LIMIT 5;
'''
q(query)

,publisher,books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


`"Penguin Books" - издательство, которое выпустило наибольшее число книг (42 книги)`

**4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [22]:
query = '''
SELECT a.author,
       ROUND(AVG(q.avg_rating), 2) AS average_rating
FROM (
    SELECT b.author_id,
           b.title,
           AVG(r.rating) AS avg_rating
    FROM books b
    LEFT JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.title, b.author_id
    HAVING COUNT(r.rating_id) >= 50
) q
LEFT JOIN authors a ON q.author_id = a.author_id
GROUP BY a.author
ORDER BY average_rating DESC
LIMIT 5;
'''
q(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Arthur Golden,4.11


`"J.K. Rowling/Mary GrandPré" - авторы с самой высокой средней оценкой (4.28)`

**5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок**

In [23]:
query = '''
SELECT ROUND(AVG(s.total_reviews), 2) AS avg_review_count
FROM (
    SELECT r.username,
           COUNT(r.review_id) AS total_reviews
    FROM reviews r
    WHERE r.username IN (
        SELECT ra.username
        FROM ratings ra
        GROUP BY ra.username
        HAVING COUNT(ra.rating_id) > 48
    )
    GROUP BY r.username
) s;
'''
q(query)

,avg_review_count
0,24.0


`24 обзора- среднее количество обзоров от пользователей, которые поставили больше 48 оценок.`

### Выводы
1. После 1 января 2000г вышло 819 книг.
2. "Twilight" - книга, которая имеет наибольшее количество обзоров (7 обзоров)
3. "Penguin Books" - издательство, которое выпустило наибольшее число книг (42 книги)
4. "J.K. Rowling/Mary GrandPré" - авторы с самой высокой средней оценкой (4.28)
5. 24 обзора- среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Рекомендации для бизнеса
1. Современные книги: Учитывая, что после 1 января 2000 года вышло 819 книг, стоит обратить внимание на современные книги. Они могут привлечь больше читателей, так как они отражают актуальные темы и тенденции. 
2. Популярные книги: Книга “Twilight” имеет наибольшее количество обзоров (7 обзоров). Это указывает на ее популярность среди читателей. Предлагая такие книги, вы можете привлечь больше читателей.  
3. Издательства с хорошей репутацией: “Penguin Books” выпустило наибольшее число книг (42 книги). Сотрудничество с издательствами, которые имеют хорошую репутацию и выпускают большое количество книг, может увеличить доверие читателей.
4. Авторы с высокими оценками: “J.K. Rowling/Mary GrandPré” - авторы с самой высокой средней оценкой (4.28). Предлагая книги от авторов с высокими оценками, вы можете увеличить удовлетворенность читателей.  
5. Активные пользователи: Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора. Это показывает, что активные пользователи вносят значительный вклад в общее количество обзоров. Поощрение активных пользователей может помочь увеличить общее количество обзоров и оценок.  
